In [157]:
import gc
import sys
import os
from tqdm import tqdm

import numpy as np
import pandas as pd #!#
import random
from einops import rearrange, repeat, reduce

from joblib import Parallel, delayed # https://www.notion.so/joblib-da8f5ee8dbd44da19b36da04bd657bb1
from torch.utils.data import Dataset, DataLoader

import tez # https://www.notion.so/Tez-093f1f31cba646e3963108294563ddd1
from tez.callbacks import Callback
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoConfig, AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup
from sklearn import metrics

gc.enable()
sys.path.append("../input/tez-lib/") #!#
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.simplefilter("ignore", UserWarning)

In [158]:
df = pd.read_csv(os.path.join(os.getcwd(), f'input/train_{5}folds.csv')) 

training_samples = []

for idx in df['id'].unique():
    filename = os.path.join('./input', "train", idx + ".txt")
    with open(filename, "r") as f:
        text = f.read()
        
    encoded_text = tokenizer.encode_plus(
        text,
        add_special_tokens=False,
        return_offsets_mapping=True,
    )
    
    input_ids = encoded_text["input_ids"]
    offset_mapping = encoded_text["offset_mapping"]
    
    sample = {
        "id": idx,
        "input_ids": input_ids,
        "text": text,
        "offset_mapping": offset_mapping,
    }
    
    # token_type_ids 만들기
    token_type_ids_list = []
    processed_text, processed_idx_list = _replace_awkend(text) # nltk 를 위한 preprocessing
    start_idx_list, end_idx_list = _extract_sentence_idx(processed_text) # 문장 index 추출하기
    for start_idx, end_idx in zip(start_idx_list, end_idx_list):
        text_type_ids = [0] * len(text)
        text_type_ids[start_idx:end_idx] = [1] * (start_idx - end_idx) #!# start_idx:end_idx 로 잘 추출되는지 확인
        
        token_type_ids = []
        for i, (offset1, offset2) in enumerate(encoded_text["offset_mapping"]):
            if any(text_type_ids[offset1:offset2]): # 1개의 text 라도 0 이외의 값
                if len(text[offset1:offset2].split()) > 0:
                    token_type_ids.append(1)
                    
            else:
                token_type_ids
        
        token_type_ids_list.append()
        
    for _, row in temp_df.iterrows():
        text_labels = [0] * len(text)
        discourse_start = int(row["discourse_start"])
        discourse_end = int(row["discourse_end"])
        prediction_label = row["discourse_type"]
        text_labels[discourse_start:discourse_end] = [1] * (discourse_end - discourse_start)
        target_idx = []
        for map_idx, (offset1, offset2) in enumerate(encoded_text["offset_mapping"]):
            if sum(text_labels[offset1:offset2]) > 0:
                if len(text[offset1:offset2].split()) > 0:
                    target_idx.append(map_idx)

        targets_start = target_idx[0]
        targets_end = target_idx[-1]
        pred_start = "B-" + prediction_label
        pred_end = "I-" + prediction_label
        input_labels[targets_start] = pred_start
        input_labels[targets_start + 1 : targets_end + 1] = [pred_end] * (targets_end - targets_start)
        sample["input_ids"] = input_ids
        sample["input_labels"] = input_labels
        training_samples.append(sample)
        
        
    if len(processed_idx_list) != 0:
        start_idx_list = _postprocess_sent_idx(start_idx_list, processed_idx_list)
        end_idx_list = _postprocess_sent_idx(end_idx_list, processed_idx_list)
    
    # input_label 만들기
    input_labels = copy.deepcopy(input_ids)
    for i, _ in enumerate(input_labels):
        input_labels[i] = "O"
        
    temp_df = df[df['id'] == idx]
    for _, row in temp_df.iterrows():
        

In [ ]:
    
        temp_df = df[df["id"] == idx]
        for _, row in temp_df.iterrows():
            text_labels = [0] * len(text)
            discourse_start = int(row["discourse_start"])
            discourse_end = int(row["discourse_end"])
            prediction_label = row["discourse_type"]
            text_labels[discourse_start:discourse_end] = [1] * (discourse_end - discourse_start)
            target_idx = []
            for map_idx, (offset1, offset2) in enumerate(encoded_text["offset_mapping"]):
                if sum(text_labels[offset1:offset2]) > 0:
                    if len(text[offset1:offset2].split()) > 0:
                        target_idx.append(map_idx)

            targets_start = target_idx[0]
            targets_end = target_idx[-1]
            pred_start = "B-" + prediction_label
            pred_end = "I-" + prediction_label
            input_labels[targets_start] = pred_start
            input_labels[targets_start + 1 : targets_end + 1] = [pred_end] * (targets_end - targets_start)
        sample["input_ids"] = input_ids
        sample["input_labels"] = input_labels
        training_samples.append(sample)

In [233]:
for text in text_list:
    # text = text_list[3]
    processed_text, processed_idx_list = _replace_awkend(text)
    assert len(processed_idx_list) == 0

AssertionError: 

In [ ]:
def _replace_awkend(text):
    '''"문장.문장", "문장 .문장" 을 "문장. 문장" 으로 바꿔준다.
        Parameters
            - text (str) : "문장 .문장", "문장.문장"
        Return
            - text (str) : "문장. 문장"
        
    nltk 의 nltk.sent_tokenize() 는 문장    
    cf) "U.S. gov" 를 "U. S. gov" 로 바꾸지만, nltk 는 다행히 후자를 하나의 문장으로 취급한다.
    '''
    # "문장 .문장"
    text = re.sub(r' \.', r'. ', text) 
    
    # "문장.문장"
    replace_token = re.findall(r'\w\.\w', text) 
    replace_idx = [text.index(token) + i + 1 for i, token in enumerate(replace_token)]
    for idx in replace_idx:
        text = text[:idx] + '. ' + text[idx+1:]        
    
    return text, replace_idx

In [229]:
# extract sentence index
def _extract_sentence_idx(text):
    '''nltk 를 활용해서 문장을 추출한다.
        Parameters
            - text : nltk 가 오작동하지 않도록 전처리된 자료
        Returns
            - start_idx_list (list) : 해당 문장이 시작하는 index
            - end_idx_list (list) : 해당 문장이 끝나는 index
        
        Assert
            - 각 i 에 대해서 text[start_idx_list[i]:end_idx_list[i]] 는 
                하나의 문장에 대응한다.
    '''
    sent_list = nltk.sent_tokenize(text)
    
    start_idx_list = []
    end_idx_list = []
    for i, sent in enumerate(sent_list):
        start_idx_list.append(text.index(sent))
        end_idx_list.append(start_idx_list[-1] + len(sent))

    for i, _ in enumerate(sent_list):
        assert text[start_idx_list[i]:end_idx_list[i]] == sent_list[i]
        
    return start_idx_list, end_idx_list

In [300]:
import copy
import warnings

def _postprocess_sent_idx(sent_idx_list, processed_idx_list):
    postprocess_sent_idx_list = copy.deepcopy(sent_idx_list)
    for i, sent_idx in enumerate(sent_idx_list):
        for processed_idx in processed_idx_list:
            if sent_idx > processed_idx:
                postprocess_sent_idx_list[i] -= 1
                
    # test code : 추출 목표가 추출 결과와 일치하지 않으면 그 부분을 반환한다.
    diff_list = []
    for i in range(len(start_idx_list)):
        start_idx = start_idx_list[i]
        end_idx = end_idx_list[i]
        original_start_idx = original_start_idx_list[i]
        original_end_idx = original_end_idx_list[i]

        if processed_text[start_idx:end_idx] != text[original_start_idx:original_end_idx]:
            diff_list.append([processed_text[start_idx:end_idx], text[original_start_idx:original_end_idx]])

    if len(diff_list) != 0:
        for diff in diff_list:
            print(f"{diff[0]}\n{diff[1]}")
        
    return postprocess_sent_idx_list

original_start_idx_list = _postprocess_sent_idx(start_idx_list, processed_idx_list)
original_end_idx_list = _postprocess_sent_idx(end_idx_list, processed_idx_list)

# test code : 추출 목표가 추출 결과와 일치하는가?
diff_list = []
for i in range(len(start_idx_list)):
    start_idx = start_idx_list[i]
    end_idx = end_idx_list[i]
    original_start_idx = original_start_idx_list[i]
    original_end_idx = original_end_idx_list[i]
    
    if processed_text[start_idx:end_idx] != text[original_start_idx:original_end_idx]:
        diff_list.append([processed_text[start_idx:end_idx], text[original_start_idx:original_end_idx]])
    
if len(diff_list) != 0:
    for diff in diff_list:
        print(f"{diff[0]}\n{diff[1]}")
    warnings.warn("exist different sentence (processed, original)"+f"{diff_list}")

In a recent article I read it said "2017 U. S. Cell Phone and Driving Statistics.
In a recent article I read it said "2017 U.S. Cell Phone and Driving Statistics.


In [305]:
target_id_map = {
    "B-Lead": 0,
    "I-Lead": 1,
    "B-Position": 2,
    "I-Position": 3,
    "B-Evidence": 4,
    "I-Evidence": 5,
    "B-Claim": 6,
    "I-Claim": 7,
    "B-Concluding Statement": 8,
    "I-Concluding Statement": 9,
    "B-Counterclaim": 10,
    "I-Counterclaim": 11,
    "B-Rebuttal": 12,
    "I-Rebuttal": 13,
    "O": 14,
    "PAD": -100,
}

id_target_map = {v: k for k, v in target_id_map.items()}

In [ ]:
# token offset -> start, end

In [321]:
import torch
a = torch.randint(0, 14, (1, 100))

In [324]:
b = [id_target_map[int(p)] for p in a[0][1:]]

In [327]:
b[0][2:]

'Lead'

In [263]:
idx = 0
phrase_preds = []
while idx < len(b):
    start, _ = offset_mapping[idx]
    if preds[idx] != "O":
        label = preds[idx][2:] #!# [2:] 는 왜 붙은걸까
    else:
        label = "O"
    phrase_scores = []
    phrase_scores.append(sample_pred_scores[idx])
    
    # label을 추출... 
    
    idx += 1
    while idx < len(offset_mapping):
        if label == "O":
            matching_label = "O"
        else:
            matching_label = f"I-{label}"
        if preds[idx] == matching_label:
            _, end = offset_mapping[idx]
            phrase_scores.append(sample_pred_scores[idx])
            idx += 1
        else:
            break # 해당 token 이 끝났기때문에 멈춘다.

    if "end" in locals(): # _, end = offset_mapping[idx]
        phrase = sample_text[start:end]
        phrase_preds.append((phrase, start, end, label, phrase_scores))

1914

In [ ]:
# start, end -> split index
temp_df = []
for phrase_idx, (phrase, start, end, label, phrase_scores) in enumerate(phrase_preds):
    word_start = len(sample_text[:start].split())
    word_end = word_start + len(sample_text[start:end].split())
    word_end = min(word_end, len(sample_text.split()))
    ps = " ".join([str(x) for x in range(word_start, word_end)])
    
    # label 이 O 가 아닌 친구들만 temp_df 에 넣는다.
    if label != "O":
        if sum(phrase_scores) / len(phrase_scores) >= proba_thresh[label]:
            temp_df.append((sample_id, label, ps))


In [255]:
    for processed_idx in processed_idx_list:
        if sent_idx > processed_idx:
            a = 1

In [256]:
sent_idx

0

In [257]:
processed_idx

1136

In [247]:
sent_idx

0

In [242]:
            sent_idx_list[i] -= 1
return sent_idx_list

[1136]

In [243]:
for i, j in zip(processed_idx_list, start_idx_list):
    if i != j:
        print(i, j)

1136 0


In [215]:
processed_text[0:0+110]

"Texting and driving\n\nOver half of drivers in today's society have this horrible habit of texting and driving. "

In [207]:
for pro_text in nltk.sent_tokenize(processed_text):
    if pro_text not in processed_text:
        print(pro_text)

In [206]:
nltk.sent_tokenize(processed_text)

["Texting and driving\n\nOver half of drivers in today's society have this horrible habit of texting and driving.",
 "This has to stop immediately, texting and driving puts people's lives at risk.",
 'This simple act has caused many fatalities over that last years.',
 'Fortunately, the government have put a new law to deal with this problem with our generations problem of texting and driving.',
 'I strongly feel that texting and driving is dangerous and should be prohibited.',
 'Diving more into detail, texting and driving is hazardous and life threatening.',
 "Just imagine you're on the road driving to your friend's house and someone shoots you a text.",
 'You look down not even for 5 seconds; then you accidents swerve into the next lane hitting a car.',
 'There are so many different scenarios that could go wrong in that situation.',
 "Your life is over just because you couldn't wait to text your friend.",
 "The consequences that texting and driving comes with just isn't worth it.",
 

In [175]:
len(awk_list)

1834

In [176]:
len(text_list)

15594

In [166]:
k = 0
for i, j in zip(' '.join(nltk.sent_tokenize(text)), text):
    if i != j:
        print(k)
        break
    k += 1

400


In [182]:
def _replace_awkend(text):
    '''"문장.문장", "문장 .문장" 을 "문장. 문장" 으로 바꿔준다.
        Parameters
            - text (str) : "문장 .문장", "문장.문장"
        Return
            - text (str) : "문장. 문장"
        
    nltk 의 nltk.sent_tokenize() 는 문장    
    cf) "U.S. gov" 를 "U. S. gov" 로 바꾸지만, nltk 는 다행히 후자를 하나의 문장으로 취급한다.
    '''
    # "문장 .문장"
    text = re.sub(r' \.', r'. ', text) 
    
    # "문장.문장"
    replace_token = re.findall(r'\w\.\w', text) 
    replace_idx = [text.index(token) + i + 1 for i, token in enumerate(replace_token)]
    for idx in replace_idx:
        text = text[:idx] + '. ' + text[idx+1:]        
    
    return text, replace_idx

In [153]:
sentence_text_list = list(map(nltk.sent_tokenize, text_list))

In [156]:
nltk.sent_tokenize(sentence_text_list[0][0])

['Phones\n\nModern humans today are always on their phone.']

In [122]:
processed_text_list = list(map(_replace_awkend, text_list))

In [136]:
i = 0
for processed_text, text in zip(processed_text_list, text_list):
    if len(processed_text.split()) != len(text.split()):
        print(i)
        break
    i += 1

4


In [141]:
i = 0
for processed_text, text in zip(processed_text_list[4], text_list[4]):
    if processed_text != text:
        print(i)
        break
    i += 1

1137


In [149]:
nltk.sent_tokenize(processed_text_list[4][i-10:i+10])

['d "2017 U. S. Cell P']

In [152]:
text_list[4][i-100:i+100]

'rect Message). Texting and driving has taken many lives. In a recent article I read it said "2017 U.S. Cell Phone and Driving Statistics. In 2012, 3,328 people were killed in distraction-related crash'

In [150]:
nltk.sent_tokenize(text_list[4][i-10:i+10])

['d "2017 U.S.', 'Cell Ph']

In [134]:
a = '5 hours a day no stop.All they do'

In [135]:
a.split(), _replace_awkend(a).split()

(['5', 'hours', 'a', 'day', 'no', 'stop.All', 'they', 'do'],
 ['5', 'hours', 'a', 'day', 'no', 'stop.', 'All', 'they', 'do'])

In [111]:
import re
def _replace_awkend1(text):
    '''nltk 는 "문장 .문장" 에서 오작동한다. 바꿔주자.
        Parameters
            - text (str) : "문장 .문장"
        Return
            - text (str) : "문장. 문장"
    '''
    text = re.sub(r' \.', r'. ', text)
    assert ' .' not in text
    return text

def _replace_awkend2(text):
    '''nltk 는 "문장.문장" 에서 오작동한다. 바꿔주자.
        Parameters
            - text (str) : "문장.문장"
        Return
            - text (str) : "문장. 문장"
    '''
    replace_token = re.findall(r'\w\.\w', text) 
    replace_idx = [text.index(token) + i + 1 for i, token in enumerate(replace_token)]
    
    for idx in replace_idx:
        text = text[:idx] + '. ' + text[idx+1:]
        
    assert len(re.findall(r'\w\.\w', text)) is 0
    return text

In [63]:
_replace_awkend('문장 .문장')

'문장. 문장'

In [82]:
a = 'Modern humans today are always on their phone .\
They are always on their phone more than 5 hours a day no stop.\
All they do is text back and forward and just have group Chats on social media.\
They even do it while driving.'

In [78]:
import re
len(re.findall(r'\w\.\w', a)) is 0

False

In [72]:
f = a.index(b[0]) + 1

In [76]:
a[:f] + '. ' + a[f+1:]

'Modern humans today are always on their phone .They are always on their phone more than 5 hours a day no stop. All they do is text back and forward and just have group Chats on social media.They even do it while driving.'

In [ ]:
for pat in b:
    

In [ ]:
re.sub(r' \.', r'. ', a)

In [27]:
nltk.sent_tokenize('a. k. a.')

['a. k. a.']

In [16]:
df['discourse_text'][0]

'Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.'

In [55]:
re.sub(r' \.', r'. ', '문장 .문장')

'문장. 문장'

In [12]:
import nltk
nltk.sent_tokenize(df['discourse_text'][0])

['Modern humans today are always on their phone.',
 'They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media.',
 'They even do it while driving.']

In [ ]:
#4# preprocess and wrap into dataset class
args.model = './model'

tokenizer = AutoTokenizer.from_pretrained(args.model) #!# 나중에 local 에서 불러오도록 코드 수정 
training_samples = prepare_training_data(train_df, tokenizer, args, num_jobs=NUM_JOBS)
valid_samples = prepare_training_data(valid_df, tokenizer, args, num_jobs=NUM_JOBS)

train_dataset = FeedbackDataset(training_samples, args.max_len, tokenizer)